In [10]:
#Cargamos las librerías necesarias para el análisis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import time 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import os
from pathlib import Path
from glob import glob
import pathlib
current_dir = '/home/floayza/Documentos/lina/Francis_data/Baseline_fmri/HC'
all_sub_dir_paths = glob(str(current_dir) + '/*/') 
all_sub_dir_paths
path = pathlib.PurePath(all_sub_dir_paths[0])

In [7]:
path

PurePosixPath('/home/floayza/Documentos/lina/Francis_data/Baseline_fmri/HC/Habits_57')

In [17]:
import os
from pathlib import Path
from glob import glob
import pathlib
from tsfresh import extract_features
import pathlib
from scipy.io import loadmat
data=[]
wdata=[]
current_dir = '/home/floayza/Documentos/lina/Francis_data/Baseline_fmri/HC'
group=1 # los grupos están organizados por carpetas. en este caso HC = grupo 1
all_sub_dir_paths = glob(str(current_dir) + '/*/') 
all_sub_dir_paths
i=0
feat=[]
for file in all_sub_dir_paths:
    print(file)
    folder = pathlib.PurePath(file)
#    folder='/home/floayza/Documentos/lina/Francis_data/Baseline_fmri/HC/Habits_08//'
    basemat='*.mat'
    i=0
    for file in sorted(glob(os.path.join(folder, basemat))):# recorremos y cargamos cada archivo .mat
        dat=loadmat(file) # cargamos el archivo
        thePoints=dat['S']['thePoints'][0][0] # seleccionamos los datos 'x' y 'y'
        Sbuttons=dat['S']['Sbuttons'] # seleccionamos el bloque o ventana de datos
        Ts=dat['S']['Ts'][0][0][0] # seleccionamos el tiempo
        trial=dat['S']['trial_cond'][0][0][i][0] # seleccionamos la etiqueta 'cond'
        length=len(Sbuttons[0][0])
        rule = Ts[-1]/length
        time = np.transpose(np.arange(start=0, stop=Ts[-1]+rule, step=rule))
        time = time[0:len(thePoints[:,0])] # make same matrix size
        data=[time, thePoints[:,0], thePoints[:,1], trial]
        data=np.transpose([time, thePoints[:,0], thePoints[:,1], trial])
    #    data = data[(data[1] != 683) & (data[2] != 384)]
    #    df=pd.DataFrame.transpose(pd.DataFrame(data))
        wdata.append(data)
        i=i+1
#        data=[]
        #print(file,i)
        df=pd.DataFrame(wdata)
        df.rename(columns={0: "time", 1: "X", 2: 'Y', 3: 'cond'}, inplace=True) # colocamos nombre a las columnas
        features=[]
        dfe=[]
        feat=[]
# Para la extracción de features, clasifico y concateno las matrices de todos los grupos por eventos.
# df es el dataframe final
    for event in range(40): # son 40 repeticiones en total por sujeto
    #    ndf=df[(df['cond'] == event+1)]
        print(event)
                    #break
    #    if ndf.shape[0]>0:
    #    for i in range(15):
    #    dfe=df['X'][event],df['Y'][event]
        dfe = pd.DataFrame(columns=['id', 'time','x','y']) # creo un nuevo df con esas columnas
        dfe['id']=np.ones(len(df['time'][event]),dtype=int) # id siempre el mismo para cada tramo
        dfe['time']=df['time'][event] # reseteo el tiempo para cada tramo
        dfe['x']=df['X'][event]
        dfe['y']=df['Y'][event]
        # delete origin values
        dfe = dfe[(dfe.x != 683) & (dfe.y != 384)]                   
        # extraemos las características
        features = extract_features(dfe, column_id="id", column_sort="time")# extrae las 780 características
        # añadimos las etiquetas. Recuerden que Python inicia desde '0', por eso sumamos 1.
        features['event'] = df['cond'][event]#añadimos la etiqueta 'event' a cada fila de características
        features['group'] = group #añadimos la etiqueta 'group' a cada fila de características
        if (event)==0:#+subject+hand+group
           feat=features
        else:
           feat=pd.concat([feat, features], ignore_index=True) # añadimos a la matriz inicial una fila por iteración.
        path = pathlib.PurePath(folder)
        feat.to_csv(os.path.join(folder,path.name+'.'+'csv')) # save csv file in directory
        data=[]

/home/floayza/Documentos/lina/Francis_data/Baseline_fmri/HC/Habits_57/
0


/home/floayza/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/floayza/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/floayza/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists

Feature Extraction: 100%|█████████████████████████| 2/2 [00:00<00:00, 11.55it/s]


1


Feature Extraction: 100%|█████████████████████████| 2/2 [00:00<00:00, 10.96it/s]


...

Feature Extraction: 100%|█████████████████████████| 2/2 [00:00<00:00, 14.51it/s]


38


Feature Extraction: 100%|█████████████████████████| 2/2 [00:00<00:00, 21.01it/s]


39


Feature Extraction: 100%|█████████████████████████| 2/2 [00:00<00:00, 14.53it/s]


In [9]:
df=pd.DataFrame(wdata)
df.rename(columns={0: "time", 1: "X", 2: 'Y', 3: 'cond'}, inplace=True)
#df

In [ ]:
df

In [ ]:
dfe['id']=np.ones(len(df['time'][event]),dtype=int)

In [ ]:
# plot from raw data
a=3 # number of trial
plt.plot(df['X'][a],df['Y'][a],)
plt.ylim(df['Y'][a].min()+300,df['Y'][a].min()+50)

In [ ]:
from tsfresh import extract_features
features=[]
dfe=[]
feat=[]
# Para la extracción de features, clasifico y concateno las matrices de todos los grupos por eventos.
# Dado que hay 5 eventos diferentes y que para cada sujeto, el número de datos es diferente ya que se eliminaron
# los parpadeos y cerradas de ojos. Por tanto se concatena por evento y luego por cada evento lo dividimos
# en 10 tramos o ventanas. A cada tramo se considera una serie temporal y se extraen características por tramo.
for event in range(40):
#    ndf=df[(df['cond'] == event+1)]
    print(event)
                #break
#    if ndf.shape[0]>0:
#    for i in range(15):
#    dfe=df['X'][event],df['Y'][event]
    dfe = pd.DataFrame(columns=['id', 'time','x','y'])
    dfe['id']=np.ones(len(df['time'][event]),dtype=int) # id siempre el mismo para cada tramo
    dfe['time']=df['time'][event] # reseteo el tiempo para cada tramo
    dfe['x']=df['X'][event]
    dfe['y']=df['Y'][event]
    # delete origin values
    dfe = dfe[(dfe.x != 683) & (dfe.y != 384)]                   
    # extraemos las características
    features = extract_features(dfe, column_id="id", column_sort="time")
    # añadimos las etiquetas. Recuerden que Python inicia desde '0', por eso sumamos 1.
    features['event'] = df['cond'][event]
    if (event)==0:#+subject+hand+group
       feat=features
    else:
       feat=pd.concat([feat, features], ignore_index=True) # añadimos a la matriz inicial una fila por iteración.
    

In [ ]:
feat.shape

In [ ]:
# saving data in the corresponding directory
import pathlib
path = pathlib.PurePath(folder)
feat.to_csv(os.path.join(folder,path.name+'.'+'csv')) # save csv file in directory

In [ ]:
for event in range(3):
    ndf=df[(df['cond'] == event+1)]
    print(event)
    dfe = pd.DataFrame(columns=['id', 'time','x','y'])
    dfe['id']=np.ones(len(df['X'][event]),dtype=int) # id siempre el mismo para cada tramo
    dfe['time']=df['time'][event] # reseteo el tiempo para cada tramo
    dfe['x']=df['X'][event]
    dfe['y']=df['Y'][event]
    # delete origin values
    dfe = dfe[(dfe.x != 683) & (dfe.y != 384)]
dfe

In [ ]:
# concat variables time and coords. important: time and coords are not the same size. I dont know why?
frames=[time,thePoints]
writing_coords=pd.concat(frames, axis=1)
writing_coords.rename(columns={0:'time'}, inplace=True)
writing_coords

In [ ]:
# plot from raw data
plt.plot(writing_coords['x'],writing_coords['y'],)
plt.ylim(writing_coords['y'].min()+100,writing_coords['y'].min()-30)

In [ ]:
# delete origin values
writing_coords = writing_coords[(writing_coords.x != 683) & (writing_coords.y != 384)]

In [ ]:
# select variables
trial_cond=datat['all_trials']['S']['trial_cond']
onset_image=pd.DataFrame(datat['all_trials']['S']['onset_image'])
endtrial_time=pd.DataFrame(datat['all_trials']['S']['endtrial_time'])